<h2>Youtube Statistics & Sentiment Analysis with SQL<h2>

We'll analyze a sample of 2000 youtube videos to glean insights into various video categories for popularity and sentiment. The original data can be found [here](https://www.kaggle.com/datasets/advaypatil/youtube-statistics). I cleaned the keyword metadata with Python Pandas library prior to loading it into this notebook.

<h3>Preparing the Data<h3>

We'll start by converting the original .csv files into a SQL database. First we'll create an empty database with 2 empty tables to store the 2 csv files: the videos table with all the video data, and the comments table with all the comments associated with each video in the videos table.

In [90]:
import sqlite3
from pathlib import Path

Path("youtube.db").touch()

conn = sqlite3.connect("youtube.db")
c = conn.cursor()

In [91]:
video_table = """CREATE TABLE videos (
           title text, video_id text, 
           published_at, keyword text, 
           likes int, comments int, 
           views int
                );"""
comments_table = """CREATE TABLE comments (
              row int, video_id text, 
              comment text, likes int, sentiment int
                );"""

c.execute(video_table)
c.execute(comments_table)

Next will load the data from the csv files into the database file we just created, and then run a test query to check that the data imported correctly.

In [92]:
import pandas as pd

videos = pd.read_csv("videos.csv")
videos.to_sql("videos", conn, if_exists="append", index=False)

comments = pd.read_csv("comments.csv")
comments.to_sql("comments", conn, if_exists="append", index=False)

18409

In [93]:
test_query = """SELECT * FROM videos"""
c.execute(test_query).fetchmany(2)

[('Apple Pay Is Killing the Physical Wallet After Only Eight Years | Tech News Briefing Podcast | WSJ',
  'wAZZ-UWGVHI',
  '8/23/22',
  'tech',
  3407,
  672,
  135612),
 ('The most EXPENSIVE thing I own.',
  'b3x28s61q3c',
  '8/24/22',
  'tech',
  76779,
  4306,
  1758063)]

In [94]:
test_query_2 = """SELECT * FROM comments"""
c.execute(test_query_2).fetchmany(2)

[(None,
  'wAZZ-UWGVHI',
  "Let's not forget that Apple Pay in 2014 required a brand new iPhone in order to use it.  A significant portion of Apple's user base wasn't able to use it even if they wanted to.  As each successive iPhone incorporated the technology and older iPhones were replaced the number of people who could use the technology increased.",
  95,
  1),
 (None,
  'wAZZ-UWGVHI',
  'Here in NZ 50% of retailers don’t even have contactless credit card machines like pay-wave which support Apple Pay. They don’t like the high fees that come with these.',
  19,
  0)]

Now that we have our Youtube SQL database, let's do some analysis.

<h3>Querying Youtube Summary Statistics<h3>

Let's start by looking at the top ten most commented on & top ten most liked videos:


In [95]:
most_commented = """
SELECT title, comments, keyword
FROM videos
ORDER BY comments DESC
LIMIT 10;
"""
c.execute(most_commented).fetchall()

[('El Chombo - Dame Tu Cosita feat. Cutty Ranks (Official Video) [Ultra Music]',
  732818,
  'music'),
 ('I Opened A Restaurant That Pays You To Eat At It', 728774, 'mrbeast'),
 ('$456,000 Squid Game In Real Life!', 611757, 'mrbeast'),
 ('One Direction - History (Official Video)', 434688, 'music'),
 ('history of the entire world, i guess', 395657, 'history'),
 ('Martin Garrix - Animals (Official Video)', 335455, 'music'),
 ('PIKOTARO - PPAP (Pen Pineapple Apple Pen) (Long Version) [Official Video]',
  294428,
  'music'),
 ('The Weeknd - Save Your Tears (Official Music Video)', 270948, 'music'),
 ('I Spent 50 Hours Buried Alive', 239172, 'mrbeast'),
 ('I Gave My 100,000,000th Subscriber An Island', 224423, 'mrbeast')]

In [96]:
most_commented = """
SELECT title, likes, keyword
FROM videos
ORDER BY likes DESC
LIMIT 10;
"""
c.execute(most_commented).fetchall()

[('El Chombo - Dame Tu Cosita feat. Cutty Ranks (Official Video) [Ultra Music]',
  16445558,
  'music'),
 ('$456,000 Squid Game In Real Life!', 14259033, 'mrbeast'),
 ('Martin Garrix - Animals (Official Video)', 11025176, 'music'),
 ('Powfu - death bed (coffee for your head) (Official Video) ft. beabadoobee',
  7786057,
  'music'),
 ('The Weeknd - Save Your Tears (Official Music Video)', 6823113, 'music'),
 ('Glass Animals - Heat Waves (Official Video)', 6177588, 'music'),
 ('I Spent 50 Hours Buried Alive', 6055504, 'mrbeast'),
 ('Maroon 5 - Animals (Official Music Video)', 5743875, 'music'),
 ('history of the entire world, i guess', 5471653, 'history'),
 ('One Direction - History (Official Video)', 5400589, 'music')]

There's overlap between the most liked and commented videos, and these videos tend to mostly fall into the keywords of music & Mr.Beast content. These trends we see in our sample are confirmed by this [Statista article](https://www.statista.com/statistics/249396/top-youtube-videos-views/) that demonstrates the most popular videos on Youtube in 2023 are music videos, and this Wikipedia article showing that [Mr.Beast](https://en.wikipedia.org/wiki/List_of_most-subscribed_YouTube_channels) is the second most subscribed to Youtube channel, the first most subscribed channel being T-Series which is a music channel. 

Next let's look at the top ten most viewed categories and top ten most liked categories.  

In [103]:
most_viewed = """
SELECT keyword, views
FROM videos
GROUP BY keyword
ORDER BY views DESC
LIMIT 10;
"""
c.execute(most_viewed).fetchall()

[('mrbeast', 37550877),
 ('music', 15040548),
 ('animals', 10155588),
 ('education', 6216027),
 ('challenge', 6055668),
 ('food', 5451348),
 ('sports', 5093587),
 ('literature', 4231789),
 ('data science', 2586478),
 ('how-to', 2361410)]

In [104]:
most_liked = """
SELECT keyword, likes
FROM videos
GROUP BY keyword
ORDER BY likes DESC
LIMIT 10;
"""
c.execute(most_liked).fetchall()

[('mrbeast', 1962726),
 ('music', 275577),
 ('education', 137327),
 ('how-to', 135155),
 ('food', 112098),
 ('challenge', 85527),
 ('literature', 56906),
 ('biology', 56774),
 ('data science', 55282),
 ('google', 52147)]

As expected the two most viewed and liked categories are music & Mr.Beast content. But does this necessarily indicate a positive sentiment? We'll do a sentiment analysis at the end of the notebook to answer this question. For now we'll look at a final few more statistics. 

Most-liked Comments

In [110]:
most_liked_comments = """

WITH joined_comments AS ( 
SELECT c.video_id as id, c.comment as comment, c.likes as likes, v.title as title, v.keyword as keyword
FROM comments c
LEFT JOIN videos v ON c.video_id=v.video_id
)

SELECT keyword, comment, likes, title 
FROM joined_comments
ORDER BY likes DESC
LIMIT 5;
"""
c.execute(most_liked_comments).fetchall()

[('mrbeast',
  'Like I said in the video, subscribe if you haven’t already and you could win $10,000!',
  891372,
  '$456,000 Squid Game In Real Life!'),
 ('mrbeast',
  'Huge props to the set designers, everything was spot on!',
  469175,
  '$456,000 Squid Game In Real Life!'),
 ('mrbeast',
  'This is insanity, never done before in the history of this platform. Hats off to you Jimmy 👏',
  378181,
  '$456,000 Squid Game In Real Life!'),
 ('mrbeast',
  'Like I said in the video, subscribe if you haven’t already and you could win $10,000!',
  268266,
  'I Spent 50 Hours Buried Alive'),
 ('mrbeast',
  "Subscribe and i'll give you a car",
  260871,
  '24 Hours In The Most Haunted Place On Earth')]

The most liked comments are all associated with Mr.Beast videos. 

Next, let's take a look at the ratio of Views-to-Likes per Category:

In [123]:
ratio_category = """

WITH joined_comments AS ( 
SELECT c.video_id as id, c.comment as comment, v.likes as likes, 
       v.views as views, v.title as title, v.keyword as keyword
FROM comments c
LEFT JOIN videos v ON c.video_id=v.video_id
)

SELECT keyword, ROUND(100*CAST (likes AS FLOAT)/views, 1) AS ratio
FROM joined_comments
GROUP BY keyword
ORDER BY ratio DESC
LIMIT 5;
"""
c.execute(ratio_category).fetchall()

[('reaction', 12.6),
 ('chess', 9.7),
 ('nintendo', 9.6),
 ('trolling', 7.2),
 ('marvel', 6.3)]

Even videos with the highest ratio of likes to views, seem to have a low percentage of likes. This indicates that likes alone cannot determine whether sentiment towards a category is positive. For example, in the "reaction" category only 12.6% of people who watched the videos liked them: does this imply that the remaining 87.4% disliked the video? Or that they felt neutral about it? To answer this question we will need more context. <br>

One way to get this necessary context is by delving into a sentiment analysis of the comments associated with all the videos in each category. We will do this in the next section. <br>

<h3>Average Sentiment for Each Keyword Category<h3>

We will calculate the sentiment percentage by writing a query that fetches the frequency of positive, negative, and neutral comments for videos in each category, and divide each frequency by the total number of comments for that category. 

Our final output will be a table with each keyword, positive sentiment %, neutral sentiment %, and negative sentiment %. 

In [97]:
keyword_sentiment = """
WITH joined AS ( 
SELECT c.video_id, c.comment, c.sentiment, v.title, v.keyword
FROM comments c
LEFT JOIN videos v ON c.video_id=v.video_id
),

sentiment_count AS (
SELECT keyword, sentiment, COUNT(*) AS count
FROM joined
GROUP BY keyword, sentiment
),

positive AS (
SELECT keyword, count 
FROM sentiment_count 
WHERE sentiment="2"
),

negative AS (
SELECT keyword, count 
FROM sentiment_count 
WHERE sentiment="0"
),

neutral AS (
SELECT keyword, count 
FROM sentiment_count 
WHERE sentiment="1"
),

sentiment_table AS (
SELECT p.keyword AS keyword, 
       p.count AS positive, 
       neu.count AS neutral, 
       n.count AS negative,
       (p.count+neu.count+n.count) AS total
FROM positive p
LEFT JOIN negative n ON p.keyword=n.keyword
LEFT JOIN neutral neu ON n.keyword=neu.keyword
)

SELECT keyword, 
       ROUND(100*CAST(positive AS FLOAT)/total) as positive,
       ROUND(100*CAST(neutral AS FLOAT)/total) as neutral,
       ROUND(100*CAST(negative AS FLOAT)/total) as negative
  FROM sentiment_table
ORDER BY positive DESC
;
"""
c.execute(keyword_sentiment).fetchall()

#Output is a table with: keyword, positive sentiment %, neutral sentiment %, & negative sentiment %

[('challenge', 87.0, 10.0, 3.0),
 ('machine learning', 78.0, 19.0, 3.0),
 ('mathchemistry', 77.0, 16.0, 7.0),
 ('lofi', 76.0, 18.0, 6.0),
 ('food', 75.0, 16.0, 10.0),
 ('tutorial', 75.0, 20.0, 5.0),
 ('mrbeast', 73.0, 19.0, 8.0),
 ('data science', 71.0, 23.0, 6.0),
 ('music', 71.0, 23.0, 6.0),
 ('asmr', 70.0, 23.0, 7.0),
 ('game development', 70.0, 21.0, 9.0),
 ('literature', 70.0, 19.0, 11.0),
 ('how-to', 69.0, 21.0, 10.0),
 ('interview', 69.0, 21.0, 10.0),
 ('mukbang', 68.0, 23.0, 8.0),
 ('physics', 68.0, 26.0, 7.0),
 ('computer science', 66.0, 25.0, 9.0),
 ('tech', 65.0, 23.0, 12.0),
 ('vlog', 65.0, 24.0, 10.0),
 ('biology', 64.0, 24.0, 12.0),
 ('nintendo', 64.0, 25.0, 11.0),
 ('animals', 63.0, 25.0, 12.0),
 ('reaction', 63.0, 28.0, 9.0),
 ('sports', 63.0, 24.0, 13.0),
 ('business', 62.0, 24.0, 14.0),
 ('chess', 61.0, 28.0, 11.0),
 ('gaming', 61.0, 29.0, 10.0),
 ('movies', 61.0, 29.0, 10.0),
 ('sat', 59.0, 26.0, 16.0),
 ('xbox', 59.0, 29.0, 12.0),
 ('google', 58.0, 26.0, 16.0),
 ('c

Highest positive sentiment categories are associated with challenge (87% positive), learning (machine learning 78% and "math chemistry" 77% positive), food (75% positive), music (71% positive for music category and 76% for lo-fi which is a niche Youtube music trend), and ASMR (70% positive). This is different from the top categories for the highest like-to-view ratios.

Highest negative & neutral sentiment keywords are associated with news (cnn 55% negative, while news itself is only 47% positive, 31% neutral, and 22% negative), finance (46% positive, 31% neutral, and 24% negative), and trolling. News, finance, & trolling related topics can often tread into divisive & controversial topics, which explains the higher negative and/or neutral sentiment. 


<H3>Conclusion<H3>

While previously we saw that the 'reaction', 'chess','nintendo','trolling', and 'marvel' categories had the top 5 like-to-comments ratios, here those categories fall towards the lower end of positive sentiment, with trolling having only 48% positive sentiment. Instead, different top categories emerge when we look through the lens of comment sentiment analysis: challenge, learning, food, music, and ASMR. This shows that likes alone does not give us a complete picture. Looking at sentiment in the comments section allows us to build a more complete picture of audience & consumer sentiment.  

Similarly, while Mr.Beast videos were the most viewed and liked, and also had the most liked comments, surprisingly other categories beat out Mr.Beast in terms of positive sentiment. One potential cause for this could be that niche categories like "machine learning" and "lo-fi" are simply more likely to have a highely invested audience. It's also interesting to see non-Mr.Beast challenge videos beat out Mr.Beast content in terms of sentiment: again, it could simply be owing to a niche, loyal fan-base.